# Code To test recomender engine

## Feature Extraction Test

### Load Images

#### Load configuration

In [ ]:
#### Load Configurations ####
import json

# Load configuration from JSON file
def load_config(config_file='imageconfig.json'):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Example of loading and using the configuration
config = load_config()

vision_uri = config['AzureVisionService']['Uri']
vision_key = config['AzureVisionService']['Key']


print(f"Vision URI: {vision_uri}")
print(f"Vision Key: {vision_key}")

azure_openai_uri = config['AzureOpenAIService']['Uri']
azure_openai_key = config['AzureOpenAIService']['Key']


print(f"azure open URI: {azure_openai_uri}")
print(f"azure open Key: {azure_openai_key}")

# Load OpnAI configuration
openai_key = config['OpenAIService']['Key']

print(f"OpenAI Key: {openai_key}")


In [2]:
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures

from azure.core.credentials import AzureKeyCredential
from PIL import Image
import io

# Function to read image from path and analyze using Azure ImageAnalysisClient
def analyze_image(image_path):
    # Read image as bytes
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    # Create ImageAnalysisClient
    client = ImageAnalysisClient(
        endpoint=vision_uri,
        credential=AzureKeyCredential(vision_key)
    )

    # Analyze image (example: describe)
       # visual_features=["Description"]
    result = client.analyze(
        image_data=image_data,
    
       visual_features =[
        VisualFeatures.TAGS,
        VisualFeatures.OBJECTS,
        VisualFeatures.CAPTION,
        VisualFeatures.DENSE_CAPTIONS,
        VisualFeatures.READ,
       # VisualFeatures.SMART_CROPS,
        VisualFeatures.PEOPLE,
    ]
    )
    return result



In [28]:
# Example usage:
# All tag extraction good except the category tag as per indian context
#image_path = r"D:\Dinesh\Tech\GitHub\virtual-assistant\images\resized_image\men\M 1.jpg"
image_path = r"D:\Dinesh\Tech\GitHub\virtual-assistant\images\dinesh_image_trails\men_grey_suit.jpg"

print(f"Analyzing image: {image_path}")
result = analyze_image(image_path=image_path)
print(result)
# Print caption
if 'captionResult' in result:
    caption = result['captionResult']
    print(f"Caption: {caption['text']} (Confidence: {caption['confidence']})")

# Print tags
if 'tagsResult' in result and 'values' in result['tagsResult']:
    print("Tags:")
    for tag in result['tagsResult']['values']:
        print(f"- {tag['name']} (Confidence: {tag['confidence']})")

# Print dense captions
if 'denseCaptionsResult' in result and 'values' in result['denseCaptionsResult']:
    print("Dense Captions:")
    for dense_caption in result['denseCaptionsResult']['values']:
        print(f"- {dense_caption['text']} (Confidence: {dense_caption['confidence']})")

# Print people
if 'peopleResult' in result and 'values' in result['peopleResult']:
    print("People:")
    for person in result['peopleResult']['values']:
        bbox = person['boundingBox']
        print(f"- BoundingBox: {bbox}, Confidence: {person['confidence']}")
# Generate a caption for the image
# if result.caption:
#     print(f"Caption: {result.caption.text} (Confidence: {result.caption.confidence})")
# # Print tags
# if result.tags:
#     print("Tags:")
#     for tag in result.tags:
#         print(f"- {tag.name} (Confidence: {tag.confidence})")
# # Print objects detected in the image
# if result.objects:
#     print("Objects:")
#     for obj in result.objects:
#         print(f"- {obj.object_property} (Confidence: {obj.confidence})")
# # Print dense captions
# if result.dense_captions:
#     print("Dense Captions:")
#     for dense_caption in result.dense_captions:
#         print(f"- {dense_caption.text} (Confidence: {dense_caption.confidence})")



Analyzing image: D:\Dinesh\Tech\GitHub\virtual-assistant\images\dinesh_image_trails\men_grey_suit.jpg
{'modelVersion': '2023-10-01', 'captionResult': {'text': 'a suit with a tie', 'confidence': 0.7532808184623718}, 'denseCaptionsResult': {'values': [{'text': 'a suit with a tie', 'confidence': 0.7532808184623718, 'boundingBox': {'x': 0, 'y': 0, 'w': 600, 'h': 700}}, {'text': 'a suit with a tie', 'confidence': 0.7210917472839355, 'boundingBox': {'x': 69, 'y': 15, 'w': 440, 'h': 669}}, {'text': 'a close up of a tie', 'confidence': 0.860759437084198, 'boundingBox': {'x': 267, 'y': 106, 'w': 62, 'h': 213}}]}, 'metadata': {'width': 600, 'height': 700}, 'tagsResult': {'values': [{'name': 'clothing', 'confidence': 0.9997585415840149}, {'name': 'coat', 'confidence': 0.9702774286270142}, {'name': 'collar', 'confidence': 0.9564415216445923}, {'name': 'outerwear', 'confidence': 0.9557503461837769}, {'name': 'blazer', 'confidence': 0.9508758187294006}, {'name': 'button', 'confidence': 0.94688284397

In [3]:
# Read images from the specified directory
def read_images_from_directory(directory):
    from os import listdir
    from os.path import isfile, join

    image_files = [f for f in listdir(directory) if isfile(join(directory, f))]
    return [join(directory, f) for f in image_files if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

In [4]:
# loop through images in a directory and analyze each one and extract tags
def analyze_images_in_directory(directory):
    image_paths = read_images_from_directory(directory)
    for image_path in image_paths:
        print(f"Analyzing image: {image_path}")
        result = analyze_image(image_path=image_path)
        print(result)
        # Print caption
        if 'captionResult' in result:
            caption = result['captionResult']
            print(f"Caption: {caption['text']} (Confidence: {caption['confidence']})")

        # Print tags
        if 'tagsResult' in result and 'values' in result['tagsResult']:
            print("Tags:")
            for tag in result['tagsResult']['values']:
                print(f"- {tag['name']} (Confidence: {tag['confidence']})")

        if 'denseCaptionsResult' in result and 'values' in result['denseCaptionsResult']:
            print("Dense Captions:")
            for dense_caption in result['denseCaptionsResult']['values']:
                print(f"- {dense_caption['text']} (Confidence: {dense_caption['confidence']})")               

### Tag extraction

### Code Snippet to rename iamges

In [ ]:
# import os
# ### Rename the image name M 9.jpg as M_9.jpg
# def rename_image_file(image_path):    
#     base, ext = os.path.splitext(image_path)
#     new_name = base.replace(" ", "_") + ext
#     os.rename(image_path, new_name)
#     print(f"Renamed {image_path} to {new_name}")
#     return new_name
# image_collection = read_images_from_directory(r"D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women")
# for image_path in image_collection:
#     renamed_image_path = rename_image_file(image_path)
    #analyze_image(renamed_image_path)

Renamed D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F 1.jpg to D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F_1.jpg
Renamed D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F 10.jpg to D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F_10.jpg
Renamed D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F 11.jpg to D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F_11.jpg
Renamed D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F 12.jpg to D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F_12.jpg
Renamed D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F 13.jpg to D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F_13.jpg
Renamed D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\F 

In [8]:
import os

import os
from PIL import Image


input_folder = r"D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\Run1"  # use consistent plural naming
output_base = "resized_images"   # use consistent plural naming



# Target size (width, height)
target_size = (400, 800)

# Loop through both categories

output_folder = os.path.join(input_folder,output_base)

# Check if input folder exists
if not os.path.exists(input_folder):
    print(f"❌ Folder not found: {input_folder}")
    exit(1)

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Process each image in the folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)

    try:
        with Image.open(file_path) as img:
            img = img.resize(target_size)
            output_path = os.path.join(output_folder, filename)
            img.save(output_path)
            print(f"✅ Resized: {filename}")
    except Exception as e:
        print(f"⚠️ Error processing {filename}: {e}")


⚠️ Error processing Backup: [Errno 13] Permission denied: 'D:\\Dinesh\\Tech\\GitHub\\virtual-assistant\\images\\American\\resized_images\\women\\Run1\\Backup'
✅ Resized: F_1.jpg
✅ Resized: F_10.jpg
✅ Resized: F_11.jpg
✅ Resized: F_12.jpg
✅ Resized: F_13.png
✅ Resized: F_14.png
✅ Resized: F_15.jpg
✅ Resized: F_16.jpg
✅ Resized: F_17.jpg
✅ Resized: F_18.png
✅ Resized: F_2.jpg
✅ Resized: F_3.png
✅ Resized: F_4.png
✅ Resized: F_5.png
✅ Resized: F_6.png
✅ Resized: F_7.png
✅ Resized: F_8.png
✅ Resized: F_9.png
⚠️ Error processing resized_images: [Errno 13] Permission denied: 'D:\\Dinesh\\Tech\\GitHub\\virtual-assistant\\images\\American\\resized_images\\women\\Run1\\resized_images'


In [9]:
# Tag extraction from a directory of images
analyze_images_in_directory(r"D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\Run1\resized_images")

Analyzing image: D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\Run1\resized_images\F_1.jpg
{'modelVersion': '2023-10-01', 'captionResult': {'text': 'a woman in a blue dress', 'confidence': 0.8804439902305603}, 'denseCaptionsResult': {'values': [{'text': 'a woman in a blue dress', 'confidence': 0.8804439902305603, 'boundingBox': {'x': 0, 'y': 0, 'w': 400, 'h': 800}}, {'text': 'a woman wearing a blue dress', 'confidence': 0.8130084872245789, 'boundingBox': {'x': 93, 'y': 25, 'w': 159, 'h': 757}}, {'text': 'a blurry image of a plant', 'confidence': 0.8195161819458008, 'boundingBox': {'x': 32, 'y': 164, 'w': 84, 'h': 228}}, {'text': 'a blurry image of a plant', 'confidence': 0.7601820826530457, 'boundingBox': {'x': 340, 'y': 165, 'w': 57, 'h': 254}}, {'text': 'a woman wearing a blue jacket', 'confidence': 0.7569583654403687, 'boundingBox': {'x': 143, 'y': 35, 'w': 123, 'h': 177}}]}, 'metadata': {'width': 400, 'height': 800}, 'tagsResult': {'values': [{'name':

### Mens Tag Generation 

In [17]:
import re
import pandas as pd
from pathlib import Path, PureWindowsPath

# file_path = Path(r"D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\men\Men_Tags")  # Change this to your actual file path
file_path = Path(r"D:\Dinesh\Tech\GitHub\virtual-assistant\images\American\resized_images\women\Run1\Women_Tags")  # Change this to your actual file path

data = []
current = {}

def extract_confidence(text):
    match = re.search(r"\(Confidence:\s*([0-9.]+)\)", text)
    return float(match.group(1)) if match else None

with open(file_path, "r", encoding="utf-8") as file:
    for line in file:
        stripped = line.strip()

        if stripped.startswith("Analyzing image:"):
            if current:
                data.append(current)
            current = {
                "ImagePath": stripped.replace("Analyzing image:", "").strip(),
                "Caption": "",
                "CaptionConfidence": None,
                "Tags": [],
                "TagConfidences": []
            }

        elif stripped.startswith("Caption:"):
            caption_line = stripped.replace("Caption:", "").strip()
            current["CaptionConfidence"] = extract_confidence(caption_line)
            current["Caption"] = caption_line[:caption_line.rfind("(")].strip() if "(" in caption_line else caption_line

        elif stripped.startswith("-"):
            tag_line = stripped[1:].strip()
            confidence = extract_confidence(tag_line)
            tag = tag_line[:tag_line.rfind("(")].strip() if "(" in tag_line else tag_line
            current["Tags"].append(tag)
            current["TagConfidences"].append(round(confidence, 3) if confidence else None)

# Add last parsed item
if current:
    data.append(current)

# Add TagConfidenceMap and ImageName
for entry in data:
    entry["TagConfidenceMap"] = {
        tag: conf for tag, conf in zip(entry["Tags"], entry["TagConfidences"])
    }
    entry["ImageName"] = PureWindowsPath(entry["ImagePath"]).name

# Create DataFrame
df = pd.DataFrame(data)

# Optional: Save to file
# df.to_csv("parsed_image_tags.csv", index=False)




In [18]:
print(df[["ImageName", "Caption", "CaptionConfidence", "Tags", "TagConfidenceMap"]])

   ImageName                                       Caption  CaptionConfidence  \
0    F_1.jpg                       a woman in a blue dress           0.880444   
1   F_10.jpg                        a woman in a red dress           0.870892   
2   F_11.jpg     a woman standing in front of a white wall           0.871490   
3   F_12.jpg        a woman wearing sunglasses and a skirt           0.768047   
4   F_13.png                 a woman wearing a brown dress           0.799620   
5   F_14.png                  a woman wearing a blue dress           0.869113   
6   F_15.jpg                  a woman wearing a grey dress           0.767621   
7   F_16.jpg                      a woman in a beige dress           0.709720   
8   F_17.jpg                   a woman wearing a red dress           0.916152   
9   F_18.png                       a woman in a brown suit           0.828032   
10   F_2.jpg                      a woman in a white dress           0.886401   
11   F_3.png                

In [19]:
# gather all image names whose tags contains keyword formal
keyword = "formal"
filtered_df = df[df['Tags'].apply(lambda tags: any(keyword in tag.lower() for tag in tags))]
print(f"Images with keyword '{keyword}':")
print(filtered_df[['ImageName', 'Tags']])


Images with keyword 'formal':
   ImageName                                               Tags
6   F_15.jpg   [formal dress, sleeve, handbag, top, grey color]
7   F_16.jpg  [high heels, formal dress, day dress, pattern ...
9   F_18.png  [handbag, blazer, trousers, fashion accessory,...
10   F_2.jpg  [high heels, formal dress, white dress, blonde...
15   F_7.png  [coat, blazer, handbag, trousers, pantsuit, fo...
16   F_8.png  [coat, outerwear, blazer, formal wear, pink su...
17   F_9.png  [high heels, cocktail dress, red, formal wear,...


In [13]:
# Get the tages of user1 
user1_tags = df[df['ImageName'].str.contains('F_18', case=False, na=False)]['Tags'].tolist()
print(f"User1 Tags:\n  {user1_tags}")

User1 Tags:
  [['handbag', 'blazer', 'trousers', 'fashion accessory', 'formal dress', 'trench coat', 'top', 'tan color', 'suit', 'black bag from a swinger', 'brown hair and a tan jacket', 'tan jacket', 'brown suit']]


### Recommender engine

In [22]:
from openai import OpenAI


# Azure OpenAI API setup
# openai.api_key = azure_openai_key
# openai.api_base = azure_openai_uri

# # Sample Pandas DataFrame (this would be your wardrobe DataFrame)
# data = {
#     "ImageName": ["M 10.jpg", "M 11.jpg", "M 12.jpg"],
#     "Caption": ["a grey suit with a tie", "a red dress", "a blue blazer"],
#     "Tags": [
#         ["clothing", "coat", "blazer", "collar", "outerwear", "button", "person", "formal wear", "suit", "a grey suit with a tie"],
#         ["clothing", "dress", "red dress", "formal wear", "elegant", "evening wear", "party dress"],
#         ["clothing", "blazer", "jacket", "suit", "blue blazer", "formal wear", "professional", "business attire"]
#     ]
# }

# # Convert to DataFrame
# df = pd.DataFrame(data)

# Function to interact with Azure OpenAI's ChatGPT Turbo and get suggestions
def get_dress_recommendation(user_query: str, df: pd.DataFrame) -> str:
    # Prepare the prompt for the model
    # Based on the following wardrobe items, recommend the top 3 most suitable dress items for the query
    prompt = f"The user asked: '{user_query}'\n\n"

    # Adding the items in the DataFrame to the prompt for ChatGPT to consider
    prompt += "Here are the wardrobe items:\n"
    for index, row in df.iterrows():
        prompt += f"Item: {row['ImageName']} \t Caption:{row['Caption']} \t Tags: {', '.join(row['Tags'])}\n"
    
    # Making the request to Azure OpenAI's ChatGPT Turbo
    # response = openai.ChatCompletion.create(
    #     engine="gpt-3.5-turbo",  # or the engine name you are using in Azure
    #     prompt=prompt,
    #     max_tokens=150,
    #     temperature=0.7
    # )
    client = OpenAI(
    # This is the default and can be omitted
    api_key=openai_key,
    #api_base=azure_openai_uri, # Getting error with this line
    #api_version="2023-05-15",  # Use the appropriate API version
    )

    response = client.responses.create(
        model="gpt-4o",
        instructions="You are a fashion assistant. Based on the user's query and wardrobe items, recommend the top 3 most suitable dress items.",
        input=prompt,
        )

    #print(response.output_text)
    # Get the response text (recommendations)
    #recommendations = response.choices[0].text.strip()
    
    return response.output_text

# Example User Query
#user_query = "Recommend a dress for tomorrow's interview"
#user_query = "Recommend a dress for gala dinner"
user_query = "Recommend a dress for friends meeting"

#user_query = "whats the weather like today?"
#user_query = "share me about latest Iran and Israel war news"


# Get the recommendation from ChatGPT
recommendation = get_dress_recommendation(user_query, df)

print("Recommended Items:", recommendation)


Recommended Items: For a friends' meeting, here are the top 3 suitable dress options from your wardrobe:

1. **Item: F_14.png** – A blue casual dress with a pattern. It's stylish for street fashion and can be dressed up or down with accessories like a handbag or sandals.

2. **Item: F_4.png** – A green casual dress paired with tan heels. This option is relaxed yet chic, perfect for spending time with friends.

3. **Item: F_13.png** – A brown polka dot day dress. It's comfortable and fun, making it ideal for a friendly get-together.

These outfits provide a balance of comfort and style for your occasion.
